In [ ]:
from pynput import mouse

print("--- LISTENING FOR CLICKS ---")
print("1. Go to your browser and click the buttons you need.")
print("2. The coordinates will print below for EVERY click.")
print("3. Right-Click anywhere to STOP.")
print("----------------------------")

# We use a counter just so you can easily identify "Click 5" vs "Click 6"
click_count = 0

def on_click(x, y, button, pressed):
    global click_count
    
    # We only care when the button is PRESSED down
    if pressed:
        if button == mouse.Button.left:
            click_count += 1
            print(f"Click #{click_count}: X={x}, Y={y}")
            
        elif button == mouse.Button.right:
            print("Right-click detected. Stopping...")
            return False  # This stops the listener

# Start the listener
with mouse.Listener(on_click=on_click) as listener:
    listener.join()

In [ ]:
import webbrowser
import pandas as pd
import pyautogui
import time
import random

# --- CONFIGURATION ---
# REPLACE WITH YOUR CAPTURED COORDINATES
EDIT_BTN_X, EDIT_BTN_Y = 1190.5411376953125, 453.4233703613281    
SPOILER_X, SPOILER_Y   = 701.867919921875, 746.5252075195312    
SAVE_BTN_X, SAVE_BTN_Y = 1159.8826904296875, 909.0016479492188      

# The image of the CHECKED box + Text "Contains spoilers"
REFERENCE_IMAGE = "spoiler_on.png"

# Path to your CSV that contains all the Letterboxd review URLs in a column named "Letterboxd URI"
CSV_PATH = "reviews.csv" 

# Load the CSV
try:
    df = pd.read_csv(CSV_PATH)
    urls = df['Letterboxd URI'].tolist()
    print(f"Found {len(urls)} reviews.")
except Exception as e:
    print(f"Error reading CSV: {e}")
    exit()

# Safety Failsafe: Drag mouse to top-left corner to KILL script
pyautogui.FAILSAFE = True

print("-------------------------------------------------")
print("Starting in 10 seconds...")
print("Please switch to your browser window now and leave it open.")
print("-------------------------------------------------")
time.sleep(10)

# --- 3. THE LOOP ---
count_spoiler_added = 0
count = 0
for url in urls:
    count += 1
    print(f"[{count}/{len(urls)}] Processing: {url}")
    
    # A. Open URL in default browser
    webbrowser.open(url)
    
    # B. Wait for page load (Increase if internet is slow)
    time.sleep(3)
    
    # C. Click "Edit"
    pyautogui.click(EDIT_BTN_X, EDIT_BTN_Y)
    time.sleep(2) # Wait for the edit page to load
    
    # D. CHECK: Is the spoiler box already checked?
    try:
        found = pyautogui.locateOnScreen(REFERENCE_IMAGE, confidence=0.999)
    except pyautogui.ImageNotFoundException:
        found = None

    if found:
        print("  --> [SKIP] Already has spoiler tag.")
        #pyautogui.click(SAVE_BTN_X, SAVE_BTN_Y)
        
    else:
        print("  --> [ACTION] Box is empty. Clicking it!")
        # Click the spoiler box
        pyautogui.click(SPOILER_X, SPOILER_Y)

        time.sleep(1.5)
        
        # Click Save
        pyautogui.click(SAVE_BTN_X, SAVE_BTN_Y)
        time.sleep(1)
        count_spoiler_added += 1
    
    # F. Small random pause to look human
    time.sleep(random.uniform(2, 4))
    # E. Cleanup: Close the tab
    
    pyautogui.hotkey('command', 'w') 
    
    time.sleep(0.5)

    #break

print(f"Done! Spoiler tag added to {count_spoiler_added} reviews out of {len(urls)}.")